<h1 style="text-align: center;">Forecasting traffic in Paris</h1>


In [4]:
import numpy as np
import pandas as pd
from prophet import Prophet
import sys

sys.path.append('/Users/clovispiedallu/Library/Mobile Documents/com~apple~CloudDocs/Documents/Machine Learning/Challenge/BCG/Datathon_BCG_colissiMONSTRE/datathon_bcg/module')

from dataprep import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_convention_raw = load_traffic_data(arc='convention')
df_champs_raw = load_traffic_data(arc='champs')
df_sts_raw = load_traffic_data(arc='sts')

df_convention_clean = traiter_donnees(df=df_convention_raw, arc='convention')
df_champs_clean = traiter_donnees(df_champs_raw, arc='champs')
df_sts_clean = traiter_donnees(df_sts_raw, arc='sts')

loading data for convention [...]
loading data for champs [...]
loading data for sts [...]


### External sources

In [6]:
df_worksites_raw = load_worksites()
df_worksites_clean = clean_worksites_data(df=df_worksites_raw)
df_worksites_clean

loading data for worksites [...]


,date_debut,date_fin,niveau_perturbation,voie,impact_circulation
35,2022-11-03,2023-04-14,2.0,Avenue des Champs Elysées,RESTREINTE
89,2023-04-03,2023-06-16,NaN,rue de la Convention,RESTREINTE
97,2021-07-12,2023-05-31,2.0,40 rue des Sts-Pères,RESTREINTE
159,2022-07-04,2023-05-12,1.0,Rue de la Convention,RESTREINTE
188,2022-06-13,2023-12-01,2.0,Avenue des Champs Elysées,RESTREINTE
190,2023-02-27,2023-04-28,2.0,Avenue des Champs Elysées,RESTREINTE
